In [1]:
import oracledb

connection = oracledb.connect(
    user = 'BD150224213',
    password = 'Hhqnm9',
    dsn = '172.16.12.14/xe',
)


In [2]:
cursor = connection.cursor()


In [26]:
cursor = connection.cursor()
cursor.execute('DROP TABLE produto')

cursor.execute("""
                CREATE TABLE produto(
                nome  VARCHAR2(255) NOT NULL ,
                descricao VARCHAR2(255),
                codigo VARCHAR2(30) NOT NULL PRIMARY KEY,
                custo INTEGER NOT NULL,
                custoFixo INTEGER NOT NULL,
                comissao INTEGER NOT NULL,
                imposto INTEGER NOT NULL,
                rentabilidade NUMBER NOT NULL 
                )"""
)

In [27]:
cursor = connection.cursor()

cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caneta', 'Caneta Profissional', '1', 36, 15, 5, 12, 20)
""")
cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Lapis', 'Preto B2', '2', 1, 1, 1, 1, 1)
""")

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Palmeiras', '3', 10, 10, 10, 10, 50)
                """)



cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'São Paulo', '4', 10, 10, 10, 10, 0)
                """)

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Corinthians', '5', 10, 10, 10, 10, -20)
                """)

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Ponte Preta', '6', 10, 30, 20, 20, 29.99)
                """)


connection.commit()



In [38]:


def exibir_menu():
    print("Menu:")
    print("1. Adicionar novo produto")
    print("2. Selecionar um produto")
    print("3. Sair")

def adicionar_novoProduto():
    print(36 * "=")
    print("\t Sistema de Cadastro")
    print(36 * "=")

    codProduto = input("Digite o código do produto: ")
    nomeProduto = input("Digite o nome do produto: ")
    descProduto = input("Adicione uma descrição ao produto: ")

    custoProduto = float(input("Qual o custo do Produto: "))
    custoFixoPct = float(input("Qual os custo fixos/administrativos do comércio [%]: "))
    comissaoVendaPct = float(input("Qual a comissão de venda do produto,em porcentagem [%]: "))
    impostoVendaPct = float(input("Qual a aliquota de imposto desejada [%]: "))
    margemLucroPct = float(input("Qual a margem de lucro desejada [%]: "))

    cursor = connection.cursor()

    cursor.execute(f""" 
                    INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                    VALUES ('{nomeProduto}', '{descProduto}', '{codProduto}', {custoProduto}, {custoFixoPct}, {comissaoVendaPct}, {impostoVendaPct}, {margemLucroPct})
                """)
    
    
    print('Cadastro concluído com sucesso!')


def selecionar_produto():
    # Lógica para exibir as tarefas existentes
    print("Produtos existentes:")
    print("- Tarefa 1")
    print("- Tarefa 2")

def main():
    while True:
        exibir_menu()
        
        opcao = input("Escolha uma opção: ")
        
        if opcao == "1":
            adicionar_novoProduto()
        elif opcao == "2":
            selecionar_produto()
        elif opcao == "3":
            break
        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()

Menu:
1. Adicionar novo produto
2. Selecionar um produto
3. Sair
	 Sistema de Cadastro
Cadastro concluído com sucesso!
Menu:
1. Adicionar novo produto
2. Selecionar um produto
3. Sair


In [40]:
cursor = connection.cursor()
cursor.execute('SELECT * FROM produto')
resultado = cursor.fetchall()
for row in resultado:
    print(row)
cursor.close()

('Caneta', 'Caneta Profissional', '1', 36, 15, 5, 12, 20)
('Lapis', 'Preto B2', '2', 1, 1, 1, 1, 1)
('Caderno', 'Palmeiras', '3', 10, 10, 10, 10, 50)
('Caderno', 'São Paulo', '4', 10, 10, 10, 10, 0)
('Caderno', 'Corinthians', '5', 10, 10, 10, 10, -20)
('Caderno', 'Ponte Preta', '6', 10, 30, 20, 20, 29.99)
('Teste', 'Teste', '7', 10, 10, 10, 10, 10)
